# Precision-Recall Tradeoff

In [ ]:
import pandas as pd
from sklearn import set_config
set_config(transform_output="pandas")

In [ ]:
# get dataset
titanic_df = pd.read_csv("../data/Dataset_Titanic.csv")
titanic_df.head()

In [ ]:
titanic_df.isna().sum()

In [ ]:
# prediction target
y = titanic_df["Survived"]


In [ ]:
# training data
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']
X = titanic_df[features]

X.head()

In [ ]:
# create train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
# One-hot encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
imp = SimpleImputer()

ct = ColumnTransformer(
    [('ohe', ohe, ['Sex']), 
    ('imputer', imp, ['Age'])],              
    remainder='passthrough'
)

ct.fit_transform(X_train)

In [ ]:
# create pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# define classifier (= ML-model)
clf = DecisionTreeClassifier()

# create pipeline
pipe = Pipeline([
    ('preprocessor', ct),
    ('classifier', clf)]
)

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

# Adjusting the Classification-Threshold

In [ ]:
# Source: https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/
# use model to predict values
y_pred_new = pipe.predict(X_test)
y_pred_new

In [ ]:
# get probabilities for predictions
# lists: [probability for class 0, probability for class 1]
pipe.predict_proba(X_test)

In [ ]:
# get only probabilities for class 1 ("survived")
pipe.predict_proba(X_test)[:,0]

In [ ]:
# Switching threshold from 0.5 to x (Default-threshold = 0.5)

# Only if the probability is higher than threshold, we will assign class 0 (dead). Otherwise: survived!
y_pred_new = (pipe.predict_proba(X_test)[:,0] >= 0.1).astype(int)
y_pred_new = ~y_pred_new + 2
y_pred_new

In [ ]:
print(classification_report(y_test, y_pred_new))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_new)
cm